### load data

In [1]:
from pandas import read_pickle
path = r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CZ4041 - Machine Learning\Team Project\data\full_train2.pkl"
data = read_pickle(path)
X_cols = [col for col in data.columns if col not in ["card_id","target"]]
X, y = data[X_cols], data['target']

In [2]:
import lightgbm as lgb
data = lgb.Dataset(X,y)

### commence optuna

In [15]:
import optuna

In [17]:
def objective(trial):

    params = {
        "num_leaves": trial.suggest_int('num_leaves', 193-30, 193+30),# 193
        "min_data_in_leaf": 325,
        "bagging_fraction": 0.9229024640381899, #trial.suggest_float("bagging_fraction", 0.69-0.3,0.69+0.3),
        "bagging_freq": 5, 
        "feature_fraction_bynode" : 0.5257141178498, # trial.suggest_float("feature_fraction_bynode", 0.45-0.3,0.45+0.3),
        "feature_fraction": 0.8016157141983071, # trial.suggest_float("feature_fraction", 0.56-0.3,0.56+0.3),
#         "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0.0001,0.5, log = False), # 0.05621632216128067,     
        "lambda_l1": 0.0006897572783007315, # trial.suggest_float("lambda_l1", 0.0001,0.01, log = True), # 0.00019163693019842484,
        "lambda_l2": 0.0047642304039302, # trial.suggest_float("lambda_l2", 0.0001,0.01, log = True), # 0.0027009967266884796,   
        "learning_rate": 0.01,  # trial.suggest_float("learning_rate", 0.001,0.01), # 0.01051003661177018,
        
        "boosting": "gbdt",
        "extra_trees": False,
        "objective": "regression",
        "max_depth":  -1,
        "bagging_freq": 10  ,                  # resamples rows at every k-th iteration
        "max_bin": 255 ,                        # bins continuous values
#         "device_type":  "cuda",
        "force_col_wise":  True  ,                 # reduce memory cost
        "weight_column": ""
    }
    
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "rmse")
    
    eval_hist  = lgb.cv(
        params = params, 
        train_set = data, 
        num_boost_round = 1000, 
        nfold=3, 
        metrics = ["rmse"],
        stratified = False, 
        categorical_feature = "auto",
        verbose_eval = 10,
        early_stopping_rounds = 10,
        callbacks  = [pruning_callback]
    )
    
    return min(eval_hist['rmse-mean'])

In [18]:
study = optuna.create_study(
    direction='minimize',                     
    sampler= None,          # use TPESampler if None
    pruner= None            # use median Sampler if None
)
study.optimize(objective, n_trials=30)

[I 2022-10-25 16:43:36,029] A new study created in memory with name: no-name-863c145b-9944-4b02-a1a4-03be79a77686


[LightGBM] [Info] Total Bins 80720
[LightGBM] [Info] Number of data points in the train set: 134610, number of used features: 424
[LightGBM] [Info] Total Bins 80720
[LightGBM] [Info] Number of data points in the train set: 134610, number of used features: 424
[LightGBM] [Info] Total Bins 80720
[LightGBM] [Info] Number of data points in the train set: 134610, number of used features: 424
[LightGBM] [Info] Start training from score -0.384681
[LightGBM] [Info] Start training from score -0.399389
[LightGBM] [Info] Start training from score -0.396838
[10]	cv_agg's rmse: 3.82346 + 0.0583556
[20]	cv_agg's rmse: 3.8006 + 0.0583248
[30]	cv_agg's rmse: 3.78145 + 0.0582927
[40]	cv_agg's rmse: 3.76481 + 0.0583505
[50]	cv_agg's rmse: 3.75095 + 0.0582905
[60]	cv_agg's rmse: 3.73895 + 0.0581663
[70]	cv_agg's rmse: 3.72877 + 0.0578588
[80]	cv_agg's rmse: 3.72025 + 0.0578955
[90]	cv_agg's rmse: 3.71282 + 0.0577076
[100]	cv_agg's rmse: 3.70623 + 0.0573615
[110]	cv_agg's rmse: 3.70066 + 0.0572917
[120]	c

[I 2022-10-25 16:48:50,472] Trial 0 finished with value: 3.6593246974489815 and parameters: {'num_leaves': 181}. Best is trial 0 with value: 3.6593246974489815.


[LightGBM] [Info] Total Bins 80720
[LightGBM] [Info] Number of data points in the train set: 134610, number of used features: 424
[LightGBM] [Info] Total Bins 80720
[LightGBM] [Info] Number of data points in the train set: 134610, number of used features: 424
[LightGBM] [Info] Total Bins 80720
[LightGBM] [Info] Number of data points in the train set: 134610, number of used features: 424
[LightGBM] [Info] Start training from score -0.384681
[LightGBM] [Info] Start training from score -0.399389
[LightGBM] [Info] Start training from score -0.396838
[10]	cv_agg's rmse: 3.8236 + 0.0586192
[20]	cv_agg's rmse: 3.80091 + 0.0585964
[30]	cv_agg's rmse: 3.78159 + 0.0585146
[40]	cv_agg's rmse: 3.76503 + 0.0584611
[50]	cv_agg's rmse: 3.75132 + 0.0584694
[60]	cv_agg's rmse: 3.73931 + 0.0583922
[70]	cv_agg's rmse: 3.72913 + 0.0582898
[80]	cv_agg's rmse: 3.72037 + 0.0581793
[90]	cv_agg's rmse: 3.71293 + 0.0581372
[100]	cv_agg's rmse: 3.70651 + 0.0577855
[110]	cv_agg's rmse: 3.70084 + 0.0576981
[120]	c

[I 2022-10-25 16:54:48,854] Trial 1 finished with value: 3.6600228269404695 and parameters: {'num_leaves': 191}. Best is trial 0 with value: 3.6593246974489815.


...

[I 2022-10-25 17:19:17,451] Trial 27 pruned. Trial was pruned at iteration 0.


[LightGBM] [Info] Total Bins 80720
[LightGBM] [Info] Number of data points in the train set: 134610, number of used features: 424
[LightGBM] [Info] Total Bins 80720
[LightGBM] [Info] Number of data points in the train set: 134610, number of used features: 424
[LightGBM] [Info] Total Bins 80720
[LightGBM] [Info] Number of data points in the train set: 134610, number of used features: 424
[LightGBM] [Info] Start training from score -0.384681
[LightGBM] [Info] Start training from score -0.399389
[LightGBM] [Info] Start training from score -0.396838


[I 2022-10-25 17:19:19,065] Trial 28 pruned. Trial was pruned at iteration 0.


[LightGBM] [Info] Total Bins 80720
[LightGBM] [Info] Number of data points in the train set: 134610, number of used features: 424
[LightGBM] [Info] Total Bins 80720
[LightGBM] [Info] Number of data points in the train set: 134610, number of used features: 424
[LightGBM] [Info] Total Bins 80720
[LightGBM] [Info] Number of data points in the train set: 134610, number of used features: 424
[LightGBM] [Info] Start training from score -0.384681
[LightGBM] [Info] Start training from score -0.399389
[LightGBM] [Info] Start training from score -0.396838


[I 2022-10-25 17:19:21,748] Trial 29 pruned. Trial was pruned at iteration 1.


In [19]:
study.best_trial.params, study.best_trial.value

({'num_leaves': 181}, 3.6593246974489815)

In [ ]:
({'lambda_l1': 0.0006897572783007315, 'lambda_l2': 0.0047642304039302},
 3.658316019575457)
({'bagging_fraction': 0.9229024640381899,
  'feature_fraction_bynode': 0.5257141178498,
  'feature_fraction': 0.8016157141983071},
 3.658712467143704)

In [ ]:
params = {
        "num_leaves": trial.suggest_int('num_leaves', 100, 200),
        "min_data_in_leaf": 325,
        "bagging_fraction": 0.6909615826257344, # trial.suggest_float("bagging_fraction", 0.55,0.8),
        "bagging_freq": 5, # trial.suggest_int("bagging_freq", 4,6),
        "feature_fraction_bynode" :0.4540787460650182, # trial.suggest_float("feature_fraction_bynode", 0.45-0.3,0.45+0.3),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.56-0.3,0.56+0.3),
        "min_gain_to_split": 0.05621632216128067,     
        "lambda_l1": 0.00019163693019842484,
        "lambda_l2": 0.0027009967266884796,   
        "learning_rate": 0.05,  # trial.suggest_float("learning_rate", 0.001,0.01), # 0.01051003661177018,
        
        "boosting": "gbdt",
        "extra_trees": False,
        "objective": "regression",
        "max_depth":  -1,
        "bagging_freq": 10  ,                  # resamples rows at every k-th iteration
        "max_bin": 255 ,                        # bins continuous values
#         "device_type":  "cuda",
        "force_col_wise":  True  ,                 # reduce memory cost
        "weight_column": ""
    }